In [1]:
import pickle
import pandas as pd
import h5py
import os

In [2]:
def load_robonet(metadata_path):
    df = pd.read_pickle(metadata_path, compression='gzip')
    return df

def save_metadata(df, metadata_path):
    df.to_pickle(metadata_path, compression='gzip')

robonet_root = "/media/ed/hdd/Datasets/Robonet/hdf5/"

In [3]:
metadata_path = "/media/ed/hdd/Datasets/Robonet/hdf5/meta_data.pkl"
df = load_robonet(metadata_path)

In [4]:
df["robot"].value_counts()

sawyer    68112
R3        56720
baxter    18054
franka     7873
widowx     5050
fetch      5000
kuka       1608
Name: robot, dtype: int64

In [44]:
total_viewpoints = 0
for robot in df["robot"].unique():
    print(f"{robot}'s recorded viewpoint labels")
    
    ncam = df[robot == df["robot"]]["ncam"].unique()
    print("ncam", ncam)
    
    cam_config = df[robot == df["robot"]]["camera_configuration"].nunique()
    print(cam_config)
    print(df[robot == df["robot"]]["camera_configuration"].value_counts())
    
    total_viewpoints += ncam * cam_config
print("total vp", total_viewpoints)

sawyer's recorded viewpoint labels
ncam [5]
19
vestri0                    16492
sudri0                     13338
sudri2                      6470
sudri5                      5016
vestri_table2               5015
sudri3                      3289
vestri_table3               3084
vestri_table6.5             2863
vestri_table1               2764
sudri4                      2503
vestri_table0               2390
sudri1                      1246
vestri_table5                680
vestri_table_correlated      650
vestri_table6                624
vestri1                      576
vestri_tabl2                 482
vestri_table5.5              480
vestri_table4                150
Name: camera_configuration, dtype: int64
widowx's recorded viewpoint labels
ncam [3]
5
widowx1    1468
widowx4    1188
widowx0    1185
widowx3     959
widowx2     250
Name: camera_configuration, dtype: int64
R3's recorded viewpoint labels
ncam [1]
1
google_lab    56720
Name: camera_configuration, dtype: int64
baxter's record

In [42]:
# check baxter's data

    

Checking sawyer
<KeysViewHDF5 ['env', 'file_version', 'metadata', 'misc', 'policy']>
<KeysViewHDF5 ['cam0_video', 'cam1_video', 'cam2_video', 'cam3_video', 'cam4_video', 'finger_sensors', 'high_bound', 'low_bound', 'qpos', 'qvel', 'state']>
<KeysViewHDF5 ['traj_ok']>
Checking widowx
<KeysViewHDF5 ['env', 'file_version', 'metadata', 'misc', 'policy']>
<KeysViewHDF5 ['cam0_video', 'cam1_video', 'cam2_video', 'high_bound', 'low_bound', 'qpos', 'qvel', 'state']>
<KeysViewHDF5 ['traj_ok']>
Checking R3
<KeysViewHDF5 ['env', 'file_version', 'metadata', 'misc', 'policy']>
<KeysViewHDF5 ['cam0_video', 'state']>
<KeysViewHDF5 []>
Checking baxter
<KeysViewHDF5 ['env', 'file_version', 'metadata', 'misc', 'policy']>
<KeysViewHDF5 ['cam0_video', 'cam1_video', 'cam2_video', 'finger_sensors', 'high_bound', 'low_bound', 'qpos', 'qvel', 'state']>
<KeysViewHDF5 ['traj_ok']>
Checking kuka
<KeysViewHDF5 ['env', 'file_version', 'metadata', 'misc', 'policy']>
<KeysViewHDF5 ['cam0_video', 'cam1_video', 'cam2_

In [7]:
# change baxter's camera_configuration to account for left and right viewpoints
baxter_df = df.loc["baxter" == df["robot"]]
left = ["left" in x for x in baxter_df.index]
right = [not x for x in left]

In [8]:
baxter_df[left].shape

(10125, 29)

In [9]:
baxter_df[right].shape

(7929, 29)

In [10]:
baxter_df.loc[left, "camera_configuration"] = "left_arm"
baxter_df.loc[right, "camera_configuration"] = "right_arm"

df.loc["baxter" == df["robot"]] = baxter_df
df.loc["baxter" == df["robot"], "camera_configuration"]

/home/ed/miniconda3/envs/roboaware/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


penn_baxter_left_traj0.hdf5        left_arm
penn_baxter_left_traj1.hdf5        left_arm
penn_baxter_left_traj10.hdf5       left_arm
penn_baxter_left_traj100.hdf5      left_arm
penn_baxter_left_traj1000.hdf5     left_arm
                                    ...    
penn_baxter_right_traj995.hdf5    right_arm
penn_baxter_right_traj996.hdf5    right_arm
penn_baxter_right_traj997.hdf5    right_arm
penn_baxter_right_traj998.hdf5    right_arm
penn_baxter_right_traj999.hdf5    right_arm
Name: camera_configuration, Length: 18054, dtype: object

In [11]:
# inspect Kuka iiwa's viewpoints
kuka_df = df.loc["kuka" == df["robot"]]
kuka_df.index

Index(['penn_kuka_traj0.hdf5', 'penn_kuka_traj1.hdf5', 'penn_kuka_traj10.hdf5',
       'penn_kuka_traj100.hdf5', 'penn_kuka_traj1001.hdf5',
       'penn_kuka_traj1003.hdf5', 'penn_kuka_traj1004.hdf5',
       'penn_kuka_traj1006.hdf5', 'penn_kuka_traj1007.hdf5',
       'penn_kuka_traj1008.hdf5',
       ...
       'penn_kuka_traj988.hdf5', 'penn_kuka_traj989.hdf5',
       'penn_kuka_traj99.hdf5', 'penn_kuka_traj991.hdf5',
       'penn_kuka_traj992.hdf5', 'penn_kuka_traj993.hdf5',
       'penn_kuka_traj994.hdf5', 'penn_kuka_traj996.hdf5',
       'penn_kuka_traj997.hdf5', 'penn_kuka_traj998.hdf5'],
      dtype='object', length=1608)

In [12]:
kuka_df.iloc[0 ::100]["object_batch"].tolist()

['new_kuka/collection_started_Sep_18_2019_09:30:19',
 'new_kuka3/collection_started_Sep_25_2019_09:48:30',
 'new_kuka3/collection_started_Sep_28_2019_15:21:20',
 'new_kuka3/collection_started_Sep_26_2019_10:33:08',
 'new_kuka/collection_started_Sep_18_2019_10:26:49',
 'new_kuka3/collection_started_Sep_28_2019_15:45:34',
 'new_kuka/collection_started_Sep_18_2019_21:39:02',
 'new_kuka3/collection_started_Sep_28_2019_16:56:17',
 'new_kuka/collection_started_Sep_17_2019_16:38:49',
 'new_kuka/collection_started_Sep_18_2019_18:25:29',
 'new_kuka2/collection_started_Sep_20_2019_17:34:22',
 'new_kuka2/collection_started_Sep_19_2019_10:54:53',
 'new_kuka2/collection_started_Sep_19_2019_19:10:09',
 'new_kuka2/collection_started_Sep_19_2019_19:10:09',
 'new_kuka2/collection_started_Sep_19_2019_20:53:19',
 'new_kuka2/collection_started_Sep_20_2019_15:07:16',
 'new_kuka/collection_started_Sep_18_2019_20:02:34']

In [13]:
# seems like there is new_kuka, new_kuka2, new_kuka3. Could that be the key differentiator of viewpoint?
kuka1 = ["new_kuka/" in x for x in kuka_df["object_batch"]]
kuka2 = ["new_kuka2/" in x for x in kuka_df["object_batch"]]
kuka3 = ["new_kuka3/" in x for x in kuka_df["object_batch"]]
print("kuka1", kuka_df[kuka1].shape)
print("kuka2", kuka_df[kuka2].shape)
print("kuka3", kuka_df[kuka3].shape)

kuka1_df = kuka_df[kuka1]
kuka2_df = kuka_df[kuka2]
kuka3_df = kuka_df[kuka3]

kuka1 (381, 29)
kuka2 (579, 29)
kuka3 (648, 29)


In [23]:
print(os.path.join(robonet_root, kuka1_df.index[0]))
print(os.path.join(robonet_root, kuka1_df.index[1]))
print(os.path.join(robonet_root, kuka1_df.index[100]))

# print(os.path.join(robonet_root, kuka2_df.index[0]))
# print(os.path.join(robonet_root, kuka3_df.index[0]))


/media/ed/hdd/Datasets/Robonet/hdf5/penn_kuka_traj0.hdf5
/media/ed/hdd/Datasets/Robonet/hdf5/penn_kuka_traj1.hdf5
/media/ed/hdd/Datasets/Robonet/hdf5/penn_kuka_traj189.hdf5


In [56]:
updated_metadata_path = "/media/ed/hdd/Datasets/Robonet/hdf5/meta_data.pkl"
save_metadata(df, updated_metadata_path)

In [30]:
# load ~50 baxter left viewpoint 0 trajectories
df[df["robot"] == "sawyer"]["camera_type"].value_counts()

Logitech C920    68112
Name: camera_type, dtype: int64

In [32]:
# widowx trajectories
widowx_df = df[df["robot"] == "widowx"]
widowx_subset = widowx_df[widowx_df["camera_configuration"] == "widowx1"]

In [33]:
print(os.path.join(robonet_root, widowx_subset.index[0]))
print(os.path.join(robonet_root, widowx_subset.index[50]))
print(os.path.join(robonet_root, widowx_subset.index[100]))


/media/ed/hdd/Datasets/Robonet/hdf5/berkeley_widowx_traj1000.hdf5
/media/ed/hdd/Datasets/Robonet/hdf5/berkeley_widowx_traj1170.hdf5
/media/ed/hdd/Datasets/Robonet/hdf5/berkeley_widowx_traj1300.hdf5


In [49]:
# fetch trajectories
fetch_df = df[df["robot"] == "fetch"]
fetch_df.high_bound

stanford_fetch_traj0.hdf5       NaN
stanford_fetch_traj1.hdf5       NaN
stanford_fetch_traj10.hdf5      NaN
stanford_fetch_traj100.hdf5     NaN
stanford_fetch_traj1000.hdf5    NaN
                               ... 
stanford_fetch_traj995.hdf5     NaN
stanford_fetch_traj996.hdf5     NaN
stanford_fetch_traj997.hdf5     NaN
stanford_fetch_traj998.hdf5     NaN
stanford_fetch_traj999.hdf5     NaN
Name: high_bound, Length: 5000, dtype: object